In [ ]:
import sys
import shutil
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import flopy

In [ ]:
# Path to the MODFLOW 6 repository
mf6exe = 'mf6beta'

In [ ]:
def make_grid(nlay, nrow, ncol, lx, top0, dztopbotfact=0.1):
    # dztopbotfact can be used as a multiplier on the top and bottom layer dz
    # values so these layers are relatively thin to more precisely locate the
    # top boundary
    top = np.zeros((nrow, ncol), dtype=np.float)
    botm = np.zeros((nlay, nrow, ncol), dtype=np.float)

    delr = np.round(lx / ncol, 3)
    delc = 1.
    delv = top0 / (nlay - 2)
    top[:, :] = top0 + delv * dztopbotfact
    botm[0, :, :] = top0
    
    for k in range(1, nlay):
        tp = botm[k - 1, 0, 0]
        botm[k, :, :] = tp - delv
    botm[nlay - 1, :, :] = 0. - delv * dztopbotfact
    
    delr = np.ones((ncol)) * delr
    delc = np.ones((nrow)) * delc
    return (delr, delc, top, botm)

In [ ]:
def calcdens(c, rho0=1000., drhodc=0.7):
    return rho0 + drhodc * c

In [ ]:
def get_model(ws, name, nlay, nrow, ncol, lx, top0, time_end,
              hydraulic_conductivity, specific_storage, porosity, 
              alpha_l, alpha_t,
              cfresh, csalt,
              nper=200, nstp=1,
              hstrt=None, cstrt=None,
              cncspd=None,
              diffusion_coefficient=0.,
              converge_continue=False,
              hhformulation=True,
              verbosity_level=1):

    delr, delc, top, botm = make_grid(nlay, nrow, ncol, lx, top0)

    perlen = nper * [time_end / nper]
    tdis_rc = []
    for i in range(nper):
        tdis_rc.append((perlen[i], nstp, 1.0))

    nouter, ninner = 100, 300
    hclose, rclose, relax = 1e-8, 1e-8, 0.97

    # build MODFLOW 6 files
    sim = flopy.mf6.MFSimulation(sim_name=name, version='mf6',
                                 exe_name=mf6exe,
                                 verbosity_level=verbosity_level,
                                 sim_ws=ws)
    if converge_continue:
        sim.name_file.continue_ = True
    # create tdis package
    tdis = flopy.mf6.ModflowTdis(sim, time_units='DAYS',
                                 nper=nper, perioddata=tdis_rc)

    # create gwf model
    gwfname = 'gwf_' + name
    gwtname = 'gwt_' + name
    
    gwf = flopy.mf6.ModflowGwf(sim, modelname=gwfname, newtonoptions=True)
    imsgwf = flopy.mf6.ModflowIms(sim, print_option='ALL',
                                  outer_hclose=hclose,
                                  outer_maximum=nouter,
                                  under_relaxation='DBD',
                                  inner_maximum=ninner,
                                  inner_hclose=hclose, rcloserecord=rclose,
                                  linear_acceleration='BICGSTAB',
                                  scaling_method='NONE',
                                  reordering_method='NONE',
                                  relaxation_factor=relax,
                                  no_ptcrecord=True,
                                  filename='{}.ims'.format(gwfname))
    single_matrix = False
    if single_matrix:
        sim.register_ims_package(imsgwf, [gwfname, gwtname])
    else:
        sim.register_ims_package(imsgwf, [gwfname])

    dis = flopy.mf6.ModflowGwfdis(gwf, nlay=nlay, nrow=nrow, ncol=ncol,
                                  delr=delr, delc=delc,
                                  top=top, botm=botm)

    # initial conditions
    if hstrt is None:
        hstrt = top[0, 0]
    else:
        hstrt = {'filename': 'hstrt.bin', 'factor': 1.0, 
                 'data': hstrt, 'binary': 'True'}
    ic = flopy.mf6.ModflowGwfic(gwf, strt=hstrt)

    # node property flow
    npf = flopy.mf6.ModflowGwfnpf(gwf, xt3doptions=False,
                                  save_flows=True,
                                  save_specific_discharge=True,
                                  icelltype=0,
                                  k=hydraulic_conductivity)
    
    sto = flopy.mf6.ModflowGwfsto(gwf, iconvert=0, sy=porosity, ss=specific_storage)

    hhoptions = False
    if hhformulation:
        hhoptions = [('HHFORMULATION'), ]
        hhoptions.append(('LHS'))
    buy = flopy.mf6.ModflowGwfbuy(gwf, hhoptions=hhoptions, denseref=1000., drhodc=0.7)
    
    # output control
    saverecord = {0: [('HEAD', 'LAST'), ('BUDGET', 'LAST')],
                  1: [('HEAD', 'LAST'), ('BUDGET', 'LAST')],
                  nper - 1: [('HEAD', 'LAST'), ('BUDGET', 'LAST')]}
    printrecord = {0: [('HEAD', 'ALL'), ('BUDGET', 'LAST')],
                  1: None,
                  nper - 1: [('HEAD', 'ALL'), ('BUDGET', 'LAST')]}
    oc = flopy.mf6.ModflowGwfoc(gwf,
                                budget_filerecord='{}.cbc'.format(gwfname),
                                head_filerecord='{}.hds'.format(gwfname),
                                headprintrecord=[
                                    ('COLUMNS', 10, 'WIDTH', 15,
                                     'DIGITS', 6, 'GENERAL')],
                                saverecord=saverecord,
                                printrecord=printrecord)

    # create gwt model
    gwt = flopy.mf6.MFModel(sim, model_type='gwt6', modelname=gwtname,
                            model_nam_file='{}.nam'.format(gwtname))

    if not single_matrix:
        imsgwt = flopy.mf6.ModflowIms(sim, print_option='ALL',
                                      outer_hclose=hclose,
                                      outer_maximum=nouter,
                                      under_relaxation='NONE',
                                      inner_maximum=ninner,
                                      inner_hclose=hclose, rcloserecord=rclose,
                                      linear_acceleration='BICGSTAB',
                                      scaling_method='NONE',
                                      reordering_method='NONE',
                                      relaxation_factor=relax,
                                      filename='{}.ims'.format(gwtname))
        sim.register_ims_package(imsgwt, [gwt.name])

    dis = flopy.mf6.ModflowGwtdis(gwt, nlay=nlay, nrow=nrow, ncol=ncol,
                                  delr=delr, delc=delc,
                                  top=top, botm=botm)

    # initial conditions
    if cstrt is None:
        cstrt = csalt
    else:
        cstrt = {'filename': 'cstrt.bin', 'factor': 1.0, 
                 'data': cstrt, 'binary': 'True'}
    ic = flopy.mf6.ModflowGwtic(gwt, strt=cstrt)

    # advection
    adv = flopy.mf6.ModflowGwtadv(gwt, scheme='CENTRAL')

    # dispersion
    dsp = flopy.mf6.ModflowGwtdsp(gwt, xt3d=False, diffc=diffusion_coefficient,
                                  alh=alpha_l, ath1=alpha_t)

    # mass storage and transfer
    porosity = porosity
    mst = flopy.mf6.ModflowGwtmst(gwt, porosity=porosity)
    
    # constant concentration conditions
    if cncspd is not None:
        cnc = flopy.mf6.ModflowGwtcnc(gwt, stress_period_data=cncspd)

    # output control
    saverecord = {0: [('CONCENTRATION', 'LAST'), ('BUDGET', 'LAST')],
                  1: [('CONCENTRATION', 'LAST')],
                  nper - 1: [('CONCENTRATION', 'LAST'), ('BUDGET', 'LAST')]}
    printrecord = {0: [('CONCENTRATION', 'FIRST'), ('BUDGET', 'LAST')],
                  1: None,
                  nper - 1: [('CONCENTRATION', 'LAST'), ('BUDGET', 'LAST')]}
    oc = flopy.mf6.ModflowGwtoc(gwt,
                                budget_filerecord='{}.cbc'.format(gwtname),
                                concentration_filerecord='{}.ucn'.format(
                                    gwtname),
                                concentrationprintrecord=[
                                    ('COLUMNS', 10, 'WIDTH', 15,
                                     'DIGITS', 6, 'GENERAL')],
                                saverecord=saverecord,
                                printrecord=printrecord)
    
    obslist = [['conc1', 'concentration', (1, 0, ncol // 2)],
               ['conc2', 'concentration', (nlay - 2, 0, ncol // 2)],]
    obsdict = {'{}.obs.csv'.format(gwtname): obslist}
    obs = flopy.mf6.mfutlobs.ModflowUtlobs(gwt, continuous=obsdict)

    # GWF GWT exchange
    gwfgwt = flopy.mf6.ModflowGwfgwt(sim, exgtype='GWF6-GWT6',
                                     exgmnamea=gwfname, exgmnameb=gwtname,
                                     filename='{}.gwfgwt'.format(name))

    return sim

In [ ]:
def get_results(ws, sim):
    modelnames = list(sim.model_names)
    gwfname = modelnames[0]
    gwtname = modelnames[1]
    gwf = sim.get_model(gwfname)
    gwt = sim.get_model(gwtname)

    fname = os.path.join(ws, gwfname + '.hds')
    hdobj = flopy.utils.HeadFile(fname, precision='double')
    head = hdobj.get_alldata()

    fname = os.path.join(ws, gwtname + '.ucn')
    cnobj = flopy.utils.HeadFile(fname, precision='double', text='CONCENTRATION')
    conc = cnobj.get_alldata()
    times = cnobj.get_times()
    
    fname = os.path.join(ws, gwfname + '.cbc')
    budobj = flopy.utils.CellBudgetFile(fname, precision='double')
    spdis = budobj.get_data(text='DATA-SPDIS')
    
    return head, conc, spdis, times

In [ ]:
def get_colored_patch_collection(modelgrid, head=None, ec='k', fc='k', zorder=None):
    # create patches for each cell
    xv, yv, zv = modelgrid.xyzvertices
    patches = []
    for k in range(modelgrid.nlay):
        for j in range(modelgrid.ncol):
            x0 = xv[0, j]
            x1 = xv[0, j + 1]
            y0 = zv[k, 0, j]
            y1 = zv[k + 1, 0, j]
            if head is not None:
                y0 = min(y0, head[k, 0, j])
                y0 = max(y0, modelgrid.botm[k, 0, j])
            poly = [[x0, y0], [x1, y0], [x1, y1], [x0, y1], [x0, y0]]
            #print(poly)
            patch = matplotlib.patches.Polygon(poly, closed=True, edgecolor=ec, facecolor=fc)
            patches.append(patch)
    pc = matplotlib.collections.PatchCollection(patches, zorder=zorder, match_original=True, cmap='jet')
    return pc

In [ ]:
def make_figure(gwf, head, conc, spdis, t, top0, csalt=35., 
                ax2dict=None, asoln=None, streamplot=False, vectors=False,
                figname=None):

    plt.rcParams['savefig.facecolor'] = '1.0'
    
    fig, ax = plt.subplots()
    fig.set_size_inches(10, 5)
    ax.set_aspect(1.)
    
    pxs = flopy.plot.PlotCrossSection(model=gwf, line={'row': 0}, ax=ax)
    pxs.plot_grid(alpha=1., lw=0.1)

    # draw a patch collection, with the water table as the top of the cell
    pc = get_colored_patch_collection(gwf.modelgrid, head, ec='none', fc='none', zorder=2)
    pc.set_array(conc.flatten())
    ax.add_collection(pc)

    cbar = plt.colorbar(pc, shrink=0.25)
    cbar.ax.get_yaxis().labelpad = 15
    cbar.ax.set_ylabel('CONCENTRATION, IN GRAMS PER LITER', rotation=90)
    
    # contours
    atoplot = np.ma.masked_where(head < gwf.dis.botm.array, conc)
    pxs.contour_array(atoplot, levels=np.arange(.1, 1.1, .1) * csalt, 
                      linestyles = ['-'], colors=['k'])

    if vectors:
        pxs.plot_specific_discharge(spdis, head=head, color='white',
                                    kstep=1, hstep=1, pivot='mid', minlength=0.5,
                                    scale=0.1, width=0.001, headwidth=3., headlength=5.0, 
                                    headaxislength=4.5, minshaft=0.01,
                                    zorder=10, alpha=0.50)
    
    if streamplot:
        x = gwf.modelgrid.xcellcenters
        y = gwf.modelgrid.zcellcenters
        X, Y = np.meshgrid(x, y[:, 0, 0])
        u = spdis['qx']
        u = u.reshape(X.shape)
        v = spdis['qz']
        v = v.reshape(X.shape)
        sc = ax.streamplot(X, Y, u, v, color='yellow', linewidth=0.1, arrowsize=0.35)

    # head surface
    #ax.plot(gwf.modelgrid.xcellcenters[0], head[kper, 0, 0, :])
    #plt.colorbar(qm, shrink=0.5)
    ax.set_title('TIME = {:.2f} years'.format(t / 365.))
    ax.set_xlabel('DISTANCE, IN METERS')
    ax.set_ylabel('ELEVATION, IN METERS')
    
    plt.tight_layout()
    
    if ax2dict is not None:
        with plt.rc_context({'axes.edgecolor':'white', 
                             'axes.facecolor':'yellow',
                             'xtick.color':'white', 
                             'ytick.color':'white', 
                             'font.size': 8,
                             'axes.labelcolor': 'white',
                             }):
            if 'limits' in ax2dict:
                limits = ax2dict['limits']
            else:
                limits = [0.15, 0.22, 0.2, 0.2]
            xdata = ax2dict['xdata']
            ydata = ax2dict['ydata']
            ax2 = fig.add_axes(limits)
            ax2.plot(xdata, ydata, 'k-')
            if 'xlim' in ax2dict:
                ax2.set_xlim(*ax2dict['xlim'])
            if 'ylim' in ax2dict:
                ax2.set_ylim(*ax2dict['ylim'])
            if 'xlabel' in ax2dict:
                ax2.set_xlabel(ax2dict['xlabel'])
            if 'ylabel' in ax2dict:
                ax2.set_ylabel(ax2dict['ylabel'])
    
    if figname is not None:
        plt.savefig(figname, dpi=300)
        plt.close(fig)
        
    return ax

In [ ]:
def make_movie(figfile, anmfile):
    from subprocess import check_output
    vcodec = 'libx264'
    if os.path.isfile(anmfile):
        os.remove(anmfile)
    output = check_output(['ffmpeg',
                           '-f', 'image2',
                           '-i', figfile,
                           '-vb', '20M',
                           '-vcodec', vcodec,
                           '-pix_fmt', 'yuv420p',
                           anmfile])    

In [ ]:
# Protection
sim = None
sim0 = None
gwf = None
ws = None
ws0 = None
head = None
head0 = None
conc = None
conc0 = None

# Elder

In [ ]:
rayleigh_number = 60.
rho_0 = 1000.
mu_0 = 1e-3
g = 9.81
delta_rho = 200.
drhodc = 0.7
csalt = delta_rho / drhodc * rayleigh_number / 400.
csalt = np.round(csalt, 2)
print('csalt = {}'.format(csalt))

gfact = 1.
nlay = int(52 * gfact)
nrow = 1
ncol = int(88 * gfact)
lx = 600.
lx_sz = 300.
top0 = 150.
nper = 200
nstp = 50
time_end = 73000.

# try just doing one time step
#nper = 2
#nstp = 1
#time_end = 7.3

cfresh = 0.
cstrt = 0.
porosity = 0.10
hydraulic_conductivity = 4.845e-13 * g * rho_0 / mu_0 * 24. * 3600. # m/day
hydraulic_conductivity = np.round(hydraulic_conductivity, 5)
print('hydraulic conductivity = {}'.format(hydraulic_conductivity))

specific_storage = 1.e-5 # per meter
alpha_l = 0. # meter
alpha_t = 0.0 # meter

diffusion_coefficient = (hydraulic_conductivity * top0 * (drhodc / rho_0) * (csalt - cfresh) 
                         / porosity / rayleigh_number)
diffusion_coefficient = np.round(diffusion_coefficient, 5)
print('diffusion coefficient = {}'.format(diffusion_coefficient))

rayleigh_calculated = (hydraulic_conductivity * top0 * (drhodc / rho_0) * (csalt - cfresh) 
                       / porosity / diffusion_coefficient)
print('rayleigh calculated = {}'.format(rayleigh_calculated))

In [ ]:
delr, delc, top, botm = make_grid(nlay, nrow, ncol, lx, top0)
mg = flopy.discretization.StructuredGrid(nlay=nlay, nrow=nrow, ncol=ncol,
                                         delr=delr, delc=delc, top=top, botm=botm)
pxs = flopy.plot.PlotCrossSection(modelgrid=mg, line={'row': 0})
pxs.plot_grid(alpha=1., lw=0.1)

In [ ]:
x = mg.xcellcenters[0, :]
cibd = np.zeros((nlay, nrow, ncol), dtype=np.int)
idx_sz = ((x < (lx - 1.5 * lx_sz) ) | (x > (lx - lx_sz / 2.) ))
cibd[0, 0, idx_sz] = 1
idx_sz = ((x > (lx - lx_sz) / 2.) & (x < (lx + lx_sz) / 2.))
cibd[0, 0, idx_sz] = 2
cibd[nlay - 1, 0, :] = 1
plt.imshow(cibd[:, 0, :])

# use low K for constant concentration cells
# should low K be for whole top layer?  Or just constant concentration cell?
if True:
    hydraulic_conductivity = np.ones((nlay, nrow, ncol)) * hydraulic_conductivity
    idx_sz = ((x > (lx - lx_sz) / 2.) & (x < (lx + lx_sz) / 2.))
    hydraulic_conductivity[0, :, idx_sz] = 1.e-8
    #hydraulic_conductivity[0, :, :] = 1.e-8
    hydraulic_conductivity[-1, :, :] = 1.e-8

In [ ]:
cnclist = []
for k, i, j in zip(*np.where(cibd == 1)):
    cnclist.append([(k, i, j), 0])
for k, i, j in zip(*np.where(cibd == 2)):
    cnclist.append([(k, i, j), csalt])
cncspd = {0: cnclist}

# MODFLOW 6 Run

In [ ]:
ws0 = 'temp/pxx_gwt_elder_Ra400'
name = 'elder'
sim0 = get_model(ws0, name, nlay, nrow, ncol, lx,
                 top0, time_end, 
                 hydraulic_conductivity, specific_storage, porosity, 
                 alpha_l, alpha_t,
                 cfresh, csalt, 
                 nper=nper, nstp=nstp,
                 cstrt=cstrt,
                 cncspd=cncspd,
                 diffusion_coefficient=diffusion_coefficient,
                 verbosity_level=1,
                 hhformulation=True)
sim0.write_simulation()
v = sim0.run_simulation(report=True)

In [ ]:
# load results
head0, conc0, spdis0, times0 = get_results(ws0, sim0)
gwf = sim0.get_model('gwf_elder')

In [ ]:
kper = 19
ax = make_figure(gwf, head0[kper], conc0[kper], spdis0[kper], 
                 times0[kper], top0, streamplot=True, csalt=csalt)
ax.set_xlim(300, 600)

In [ ]:
for t in [6, 20, 60, 200]:

    kper = t - 1
    ax = make_figure(gwf, head0[kper], conc0[kper], spdis0[kper], 
                     times0[kper], top0, streamplot=True, csalt=csalt)
    ax.set_xlim(300, 600)